In [30]:
!pip install openvino-dev

  Obtaining dependency information for openvino-dev from https://files.pythonhosted.org/packages/b1/2f/ce3387464d1f05a67b889f3b7bdc8cbce34224b34e4efc76bfe1a1c4f75f/openvino_dev-2024.6.0-17404-py3-none-any.whl.metadata
  Obtaining dependency information for networkx<=3.1.0 from https://files.pythonhosted.org/packages/a8/05/9d4f9b78ead6b2661d6e8ea772e111fc4a9fbd866ad0c81906c11206b55e/networkx-3.1-py3-none-any.whl.metadata
  Obtaining dependency information for openvino==2024.6.0 from https://files.pythonhosted.org/packages/6b/c4/ec78885d4046511892dc55ba3013787044381fb6e5a1fd0373b0fb7c1e98/openvino-2024.6.0-17404-cp311-cp311-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 77.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 196.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 218.3 MB/s eta 0:00:00
  Attempting uninstall: openvino
    Found existing installation: openvino 2025.0.0

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

ImportError: cannot import name 'mo' from 'openvino' (/opt/app-root/lib64/python3.11/site-packages/openvino/__init__.py)

In [4]:
# Simulated battery data
'''
data = [
    {"batteryId": 1, "stateOfCharge": 0.4384, "stateOfHealth": 98.00, "batteryCurrent": 349.57, "batteryVoltage": 397.63,
     "kmh": 234.00, "distance": 79.35, "batteryTemp": 0.00, "ambientTemp": 20.40},
    {"batteryId": 2, "stateOfCharge": 0.1234, "stateOfHealth": 95.00, "batteryCurrent": 500.00, "batteryVoltage": 350.00,
     "kmh": 200.00, "distance": 100.50, "batteryTemp": 60.00, "ambientTemp": 25.40},
    {"batteryId": 3, "stateOfCharge": 0.7684, "stateOfHealth": 99.00, "batteryCurrent": 200.00, "batteryVoltage": 400.00,
     "kmh": 180.00, "distance": 120.00, "batteryTemp": 30.00, "ambientTemp": 20.00},
]
'''

# Read the data from file
df = pd.read_csv("./data/battery_data.csv")

print(df)

                            timestamp  batteryId  ambientTemp  batteryCurrent  \
0    2025-02-12 14:26:34.190000+00:00          1        18.65           78.06   
1    2025-02-12 14:26:37.025000+00:00          1        18.36           81.42   
2    2025-02-12 14:26:40.020000+00:00          1        18.17           75.76   
3    2025-02-12 14:26:43.024000+00:00          1        18.50           72.11   
4    2025-02-12 14:26:46.023000+00:00          1        18.12           73.72   
..                                ...        ...          ...             ...   
501  2025-02-12 14:51:53.470000+00:00          1        19.29           55.85   
502  2025-02-12 14:51:56.469000+00:00          1        19.43           64.87   
503  2025-02-12 14:51:59.469000+00:00          1        19.73           55.13   
504  2025-02-12 14:52:02.469000+00:00          1        19.86           44.47   
505  2025-02-12 14:52:05.475000+00:00          1        20.05           42.55   

     batteryTemp  batteryVo

In [5]:
# Define stress condition (1 = Stress, 0 = Normal)
def detect_stress(row):
    if row["batteryCurrent"] > 400 or row["batteryTemp"] > 50 or row["stateOfCharge"] < 0.05 or row["batteryVoltage"] < 320:
        return 1  # Stress condition
    return 0  # Normal condition


# Apply stress detection
df["stressIndicator"] = df.apply(detect_stress, axis=1)

print(df)

                            timestamp  batteryId  ambientTemp  batteryCurrent  \
0    2025-02-12 14:26:34.190000+00:00          1        18.65           78.06   
1    2025-02-12 14:26:37.025000+00:00          1        18.36           81.42   
2    2025-02-12 14:26:40.020000+00:00          1        18.17           75.76   
3    2025-02-12 14:26:43.024000+00:00          1        18.50           72.11   
4    2025-02-12 14:26:46.023000+00:00          1        18.12           73.72   
..                                ...        ...          ...             ...   
501  2025-02-12 14:51:53.470000+00:00          1        19.29           55.85   
502  2025-02-12 14:51:56.469000+00:00          1        19.43           64.87   
503  2025-02-12 14:51:59.469000+00:00          1        19.73           55.13   
504  2025-02-12 14:52:02.469000+00:00          1        19.86           44.47   
505  2025-02-12 14:52:05.475000+00:00          1        20.05           42.55   

     batteryTemp  batteryVo

In [10]:
# Count the number of stress events
stress_count = df["stressIndicator"].sum()

print(f"Total stress events detected: {stress_count}")

Total stress events detected: 195


In [11]:
# Define Features and Target
features = ["stateOfCharge", "stateOfHealth", "batteryCurrent", "batteryVoltage", "kmh", "distance", "batteryTemp", "ambientTemp"]
X = df[features]
y = df["stressIndicator"]

In [14]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
# Define neural network
mlp_tf = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
mlp_tf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
# Train model
mlp_tf.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=2)

Epoch 1/50
12/12 - 2s - 135ms/step - accuracy: 0.8237 - loss: 0.5652 - val_accuracy: 0.9024 - val_loss: 0.4728
Epoch 2/50
12/12 - 0s - 4ms/step - accuracy: 0.8705 - loss: 0.4130 - val_accuracy: 0.9024 - val_loss: 0.3735
Epoch 3/50
12/12 - 0s - 4ms/step - accuracy: 0.9174 - loss: 0.3223 - val_accuracy: 0.9268 - val_loss: 0.3065
Epoch 4/50
12/12 - 0s - 4ms/step - accuracy: 0.9449 - loss: 0.2588 - val_accuracy: 0.9268 - val_loss: 0.2570
Epoch 5/50
12/12 - 0s - 4ms/step - accuracy: 0.9642 - loss: 0.2098 - val_accuracy: 0.9268 - val_loss: 0.2203
Epoch 6/50
12/12 - 0s - 5ms/step - accuracy: 0.9697 - loss: 0.1712 - val_accuracy: 0.9512 - val_loss: 0.1935
Epoch 7/50
12/12 - 0s - 4ms/step - accuracy: 0.9780 - loss: 0.1404 - val_accuracy: 0.9268 - val_loss: 0.1763
Epoch 8/50
12/12 - 0s - 4ms/step - accuracy: 0.9669 - loss: 0.1181 - val_accuracy: 0.9268 - val_loss: 0.1666
Epoch 9/50
12/12 - 0s - 4ms/step - accuracy: 0.9697 - loss: 0.1021 - val_accuracy: 0.9268 - val_loss: 0.1614
Epoch 10/50
12/12

In [19]:
# Save model as keras
mlp_tf.save("models/battery_stress_model.keras")
print("Model saved as TensorFlow format in 'models/battery_stress_model.keras'")

Model saved as TensorFlow format in 'models/battery_stress_model.keras'


In [22]:
# Load model
model = tf.keras.models.load_model("models/battery_stress_model.keras")

# Export SavedModel
tf.saved_model.save(model, "models/battery_stress_model")

print("Model converted to SavedModel format")

INFO:tensorflow:Assets written to: models/battery_stress_model/assets


INFO:tensorflow:Assets written to: models/battery_stress_model/assets


Model converted to SavedModel format


In [31]:
!mo --saved_model_dir models/battery_stress_model --output_dir models/battery_stress_model

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
2025-03-14 12:49:24.822154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-14 12:49:24.837320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-14 12:49:24.842057: E external/local_xla/xla/s